In [2]:
from Bio import AlignIO
#Alignment conducted with multiple alignment tool @ http://www.ebi.ac.uk/Tools/msa/clustalo/
aln = AlignIO.read('cpec-align.clustal', 'clustal')
print(aln[:, -1000:-200])

SingleLetterAlphabet() alignment with 8 rows and 800 columns
NGNNNNTAANTTNTNNNNNCN--CTNNNNNNNANAGTGGGAGAT...TAA vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
NGNNNNTAAN-TTANNNNTCNTN-NNNNNNNNNNNNNGGGAGAT...TAA vic-pb2-cpec-L11-f-vic-EM28_F11.ab1
NNACAATAACTTANNNNNNCNNNCNNNNN-NNNNNNNNGGAGAT...TAA vic-pb2-cpec-L1-f-vic-EM28_A11.ab1
NGANANNNNNNTNTNNNNTCGNNCNNNANGNNNNNNNNGGAGAT...TAA vic-pb2-cpec-L6-f-vic-EM28_D11.ab1
NGNCNNTAANNTNNNNNNNCGNNCNGNANGNNNNNN-NNGAGAT...TAA vic-pb2-cpec-L3-f-vic-EM28_B11.ab1
NGNCNNTAANNTNTNNNNNCGCNCNNNANNN-NNNN-NGGAGAT...TAA vic-pb2-cpec-L4-f-vic-EM28_C11.ab1
TGACAATAACTTAT----TCA--TCGTCGATGA-TGTGGGAGAT...--- A/Victoria/361/2011|KJ942687
NGACAATTACTTAT----TNN--TCGTCGANGANNGNGGGAGAT...--- vic-pb2-cpec-L10-f-vic-EM28_E11.ab1


In [19]:
aln[0].id = "L16"
aln[1].id = "L11"
aln[2].id = "L1"
aln[3].id = "L6"
aln[4].id = "L3"
aln[5].id = "L4"

L10 = aln[7]
aln[6].id = "vic"

aln[0].id = "L16"
print(aln[0].id)


L16


In [20]:
#Finding good start and end indices to trim mutant sequences at

def is_good(index, sequence):
    good = 0
    for next_base in sequence[index+1:index+100]: #100 chosen as large enough to ensure 
        if next_base != "-":
            good += 1      
    if good > 98:
        return True
def find_end(index, sequence):
    for more_index, more_base in enumerate(sequence[index:]):
        if more_base == "-" and not is_good(more_index, sequence):
            return more_index
    return False
            
start_list = []
end_list = []

for sequence in aln[0:6]:
    for index, base in enumerate(sequence):
        if base != "-":
            if is_good(index, sequence):
 
                start_list.append(index)
                if find_end(index, sequence) != False:
                    end_list.append(index + find_end(index, sequence))

                break

from collections import Counter
data = Counter(end_list)

end = data.most_common(1)[0][0] 
start = max(start_list)       
    



In [22]:
#Completing PB2 sequences for each mutant by combining with template sequence
vic_end = find_end(end, aln[6]) #Stripping the - at the end of the template sequence

full_seq_dict = {}
for sequence in aln[0:6]:
    full_seq = str(aln[6].seq)[0:start] + str(sequence.seq)[start:end] + str(aln[6].seq)[end:vic_end]
    full_seq_dict[sequence.id] = full_seq
print(full_seq_dict)

{'L16': 'AGCAAAAGCAGGTCAATTATATTCAGTATGGAAAGAATAAAAGAACTACGGAATCTGATGTCGCAGTCTCGCACTCGCGAGATACTGACAAAAACCACAGTGGACCATATGGCCATAATTAAGAAGTACACATCGGGGAGACAGGAAAAGAACCCGTCACTTAGGATGAAATGGATGATGGCAATGAAATATCCAATCACTGCTGACAAAAGGGTAACAGAAATGGTTCCGGAGAGAAATGAACAAGGACAAACTCTATGGAGTAAAATGAGTGATGCTGGATCAGATAGAGTGATGGTATCACCTTTGGCTGTAACATGGTGGAATAGGAATGGACCCGTGACAAGTACGGTCCATTACCCAAAAGTGTACAAAACTTATTTCGACAAAGTCGAAAGGTTAAAACATGGAACCTTTGGCCCTGTCCATTTCAGAAATCAAGTCAAGATACGCAGAAGAGTAGACATAAACCCTGGTCATGCAGACCTCAGTGCCAAAGAGGCACAAGATGTAATTATGGAAGTTGTTTTTCCCAATGAAGTGGGAGCCAGAATACTAACATCAGAATCACAACTAACAATAACTAAGGAGAAAAAAGAAGAGCTCCGAGATTGCAAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTACGAAAAACAAGATTTCTCCCAGTTGCTGGCGGAACAAGCAGTATATACATTGAAGTTTTACATTTGACTCAGGGAACGTGTTGGGAACAAATGTACACTCCAGGTGGAGGAGTGAGGAATGACGATGTTGACCAAAGCCTAATTATTGCGGCCAGGAACATAGTAAGAAGAGCCGCAGTATCAGCAGATCCACTAGCATCTTTATTGGAGATGTGCCACAGCACGCAAATTGGCGGAACAAGGATGGTGGACATTCTTAGACAGAACCCGACTGAAGAACAAGCTGTGGATATATGCAAGGCTGCAATGGGATTGAGAATCAGCTCAT

In [101]:
def is_diff(mutant, vic = aln[6]):
    num_mut = 0
    where_mut = []
    for base_index, base1 in enumerate(mutant[start:end]):
        if base1 != vic[base_index]:
            num_mut += 1
            where_mut.append(base_index)
    return num_mut

mut_dict = {}
seq_num = 0
for sequence in aln[0:6]:
    mut_dict[seq_num] = is_diff(sequence)
    seq_num += 1

print(mut_dict)

60